In [10]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) # makes the notebook fill the whole window

import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
#import matplotlib.patches as patches
from matplotlib import ticker
from matplotlib.collections import PatchCollection
#from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import Polygon
from sklearn.cluster import DBSCAN
import datetime
import pandas as pd
import time
import shapefile
import os
from os import listdir
from os.path import isfile, isdir, join
import CubicSplineTrack
#(c)2015 Robert Sedgewick, Kevin Wayne, and Robert Dondero
from greatcircle import great_circle
from datetime import date
import shutil
import imageio

from sklearn.cluster import DBSCAN

import warnings
warnings.filterwarnings("ignore")

#Turn off interactive plotting for pyplot
plt.ioff()

In [11]:
#------------------------#
" NEW CODE FOR MILESTONE "
#------------------------#
"""
This cell is for config variables ONLY
"""
check_minutes = 60 # minuite var
split_storm = 1 # seperate or no
is_wwlln = 0 # wwlln(1) or glm(0)
initial_data_path = "Data/"
do_all_storms = 0 # 1 for all storms, 0 for specific storms. uses storms_to_run if 0
storms_to_run = ["Arthur", "Bertha", "Beta", "Delta", "Dolly", "Edouard"]

# ncdf to csv config
glm_ncdf_data_path = "C:/Users/lance/Documents/CSP250/GLM/Storms"
glm_csv_minute_length = 60

In [4]:
#------------------------#
" NEW CODE FOR MILESTONE "
#------------------------#
"""
Cassie
Converts multiple .nc files to .csv files
For each input file, read it using GLMDataset
then save only the values we want
"""
import csv
from glmtools.io.glm import GLMDataset

def convert_ncdf_to_csv(file_paths, out_path, center):
    with open(out_path, 'w', newline='') as out_file:
        writer = csv.writer(out_file)
        writer.writerow(["Date", "Lat", "Long"])

        for in_file_path in file_paths:
            #print(f"Path: {in_file_path}")
            
            try:
                glm_data = GLMDataset(in_file_path).dataset
            except Exception as excpt:
                print(f"File at \"{in_file_path}\" has exception:")
                print(excpt)
                print(f"When being saved to \"{out_path}\"")
                continue

            
            groups = glm_data[['group_energy','group_area']]
            group_time = groups['group_time_offset'].values
            group_lon = groups['group_lon'].values
            group_lat = groups['group_lat'].values

            group_size = (glm_data['number_of_groups'].values).size
            group_indx = 0
            while group_indx < group_size:
                #print(f"Lat Center: {center['Lat']}, Long Center: {center['Long']}")
                #print(f"Lat Group: {group_lat[group_indx]}, Long Group: {group_lon[group_indx]}")
                distance_to_center = great_circle(group_lat[group_indx], group_lon[group_indx], (center['Lat']).iloc[0], (center['Long']).iloc[0])
                #print(f"Distance: {distance_to_center}")

                if ((distance_to_center <= 100 / 1.852) or ((distance_to_center >= 200 / 1.852) and (distance_to_center <= 400 / 1.852))):
                    writer.writerow([group_time[group_indx], group_lat[group_indx], group_lon[group_indx]])

                group_indx+=1
    return

#------------------------#
" NEW CODE FOR MILESTONE "
#------------------------#
"""
Cassie
Testing code for converting local .nc files to .csv files
Gets all files for ONLY one hour, devided into chunks
YOU MUST CHANGE THE PATHS TO GET THIS TO WORK
AND MAKE SURE ALL FOLDERS HAVE BEEN CREATED BEFOREHAND
"""

from os import listdir
def convert_storms():
    global storms_to_run
    if do_all_storms:
        storms_to_run = listdir(glm_ncdf_data_path)
    ncdf_storm_paths = [os.path.join(glm_ncdf_data_path, fn+'/') for fn in storms_to_run]
    csv_storm_paths = [os.path.join(initial_data_path, fn+'/') for fn in storms_to_run]
    spline_paths = [""] * len(storms_to_run)

    curr_storm_indx = 0
    while curr_storm_indx < len(csv_storm_paths):
        if not os.path.exists(csv_storm_paths[curr_storm_indx]):
            os.mkdir(csv_storm_paths[curr_storm_indx])
        
        files = listdir(csv_storm_paths[curr_storm_indx])
        file_indx = 0
        spline_found = False
        while file_indx < len(files):
            if files[file_indx].endswith("Trackfile.csv"):
                spline_paths[curr_storm_indx] = join(csv_storm_paths[curr_storm_indx]+"/", files[file_indx])
                spline_found = True
                break
            file_indx+=1

        if not spline_found:
            print("Spline Not Found For Storm " + storms_to_run[curr_storm_indx])
            return
        
        csv_storm_paths[curr_storm_indx] = os.path.join(csv_storm_paths[curr_storm_indx]+'/', "glm/")

        if not os.path.exists(csv_storm_paths[curr_storm_indx]):
            os.mkdir(csv_storm_paths[curr_storm_indx])
        
        curr_storm_indx+=1
    

    curr_storm_indx = 0
    while curr_storm_indx < len(ncdf_storm_paths):
        print(f"Start Conversion For {storms_to_run[curr_storm_indx]}")
        ncdf_day_paths = [os.path.join(ncdf_storm_paths[curr_storm_indx], fn) for fn in listdir(ncdf_storm_paths[curr_storm_indx])]
        csv_day_paths = [os.path.join(csv_storm_paths[curr_storm_indx], fn) for fn in listdir(ncdf_storm_paths[curr_storm_indx])]
        storm_center = pd.read_csv(Cubic_Spline_Path[Storm_Names.index("Hurricane " + storms_to_run[curr_storm_indx])])
        #print(Cubic_Spline_Path[Storm_Names.index("Hurricane " + (storms_to_run[curr_storm_indx])[0:-1])])
        #print(storms_to_run[curr_storm_indx])
        storm_center['Date'] = storm_center['Date'].apply(pd.to_datetime)
        start_date = storm_center['Date'][0]
        end_date = storm_center['Date'][len(storm_center)-1]
        ncdf_time_offsets = [start_date.timetuple().tm_yday - int(ncdf_day_paths[0][-3:]), start_date.hour]
        start_date = datetime.datetime(start_date.year, start_date.month, start_date.day)
        #print(ncdf_time_offsets)
        #print(center_date)
        #print(storm_center)
        breakout_flag = False
        

        curr_day_indx = ncdf_time_offsets[0]
        while curr_day_indx < len(ncdf_day_paths) and not breakout_flag:
            ncdf_hour_paths = [os.path.join(ncdf_day_paths[curr_day_indx], fn) for fn in listdir(ncdf_day_paths[curr_day_indx])]
            csv_hour_paths = [os.path.join(csv_day_paths[curr_day_indx], fn) for fn in listdir(ncdf_day_paths[curr_day_indx])]
            

            if not os.path.exists(csv_day_paths[curr_day_indx]):
                os.mkdir(csv_day_paths[curr_day_indx])
            

            if curr_day_indx == ncdf_time_offsets[0]:
                curr_hour_indx = ncdf_time_offsets[1]
            else:
                curr_hour_indx = 0
            
            while curr_hour_indx < len(ncdf_hour_paths) and not breakout_flag:
                ncdf_file_paths = [os.path.join(ncdf_hour_paths[curr_hour_indx], fn) for fn in listdir(ncdf_hour_paths[curr_hour_indx])]
                
                if not os.path.exists(csv_hour_paths[curr_hour_indx]):
                    os.mkdir(csv_hour_paths[curr_hour_indx])

                files_per_minuite = 3
                offset = glm_csv_minute_length*files_per_minuite

                center_date = start_date + datetime.timedelta(curr_day_indx, curr_hour_indx*60*60)
                center = storm_center[storm_center['Date'] == center_date]

                #convert_ncdf_to_csv(ncdf_file_paths, os.path.join(csv_hour_paths[curr_hour_indx]+'/', f'0_60.csv'), center)
                #print(center_date)
                #print(f"Date: {center_date}, Lat Center: {center['Lat']}, Long Center: {center['Long']}")
                indx = 0
                while indx < 60*files_per_minuite and not breakout_flag:
                    convert_ncdf_to_csv(ncdf_file_paths[indx:indx+offset], os.path.join(csv_hour_paths[curr_hour_indx]+'/', f'{indx/files_per_minuite}_{(indx+offset)/files_per_minuite}.csv'), center)
                    if end_date == center_date:
                        breakout_flag = True
                        break
                
                    indx += glm_csv_minute_length*files_per_minuite
                    if (indx % glm_csv_minute_length == 0):
                        center_date = start_date + datetime.timedelta(curr_day_indx, curr_hour_indx*60*60 + (indx/files_per_minuite)*60)
                        center = storm_center[storm_center['Date'] == center_date]
                        #print(f"Date: {center_date}, Lat Center: {center['Lat']}, Long Center: {center['Long']}")
                curr_hour_indx += 1
            curr_day_indx += 1
        curr_storm_indx += 1

convert_storms()

Start Conversion For Arthur
Start Conversion For Bertha
File at "C:/Users/lance/Documents/CSP250/GLM/Storms\Bertha/149\07\OR_GLM-L2-LCFA_G16_s20201490717200_e20201490717200_c20201490718138.nc" has exception:
'NoneType' object has no attribute 'groups'
When being saved to "Data/Bertha//glm/149\07/0.0_60.0.csv"


In [12]:
#-----------------------#
" UPDATED FOR MILESTONE "
#-----------------------#
"""Team WAAAAAAEEEEE Cassie"""

if do_all_storms:
    only_dirs = [d for d in listdir(initial_data_path) if isdir(join(initial_data_path, d))] # makes a list of all the folders in Data/
else:
    only_dirs = [d for d in storms_to_run]

dir_count = len(only_dirs)

WWLLN_Track_Path = [""] * dir_count # the paths of all the track files, in the same order as only_dirs
WWLLN_Locations_Path = [""] * dir_count # the paths of all the lightning locations files, in the same order as only_dirs
Cubic_Spline_Path = [""] * dir_count # the paths of all the cubic spline files, in the same order as only_dirs
glm_paths = [""] * dir_count

Image_Path = [""] * dir_count # Base Image paths
Storm_Names = [""] * dir_count # Base storm names
T_Path = [""] * dir_count

# loop thru all dirs
storms_found = 0
def load_file_paths():
    global storms_found
    dir_indx = 0
    while dir_indx < dir_count:
        
        # this gets all the file paths in a given storms' folder
        only_dirs[dir_indx] = join(initial_data_path, only_dirs[dir_indx])
        print(only_dirs)

        glm_paths[dir_indx] = only_dirs[dir_indx] + '/glm' #Lance code start here
        if not os.path.exists(glm_paths[dir_indx]):
            glm_paths[dir_indx] = '' #Lance code end here
        
        file_paths = listdir(only_dirs[dir_indx])
        
        # check if the 3 nessisary files exist
        if len(file_paths) < 2:
            only_dirs[dir_indx] = ""
            dir_indx+=1
            continue
        
        #print(only_dirs[dir_indx])
        #print(file_paths)
        
        # for every file in file_paths, sort it into its' respective arrays
        file_indx = 0
        files_found = 0
        Cubic_Found = False
        while file_indx < len(file_paths):
            # creates the files full path
            file_paths[file_indx] = join(only_dirs[dir_indx]+"/", file_paths[file_indx])
            #print(file_paths[file_indx])
            
            # sorts the file path based on the last expected word of a file and it's type
            if file_paths[file_indx].endswith("Trackfile.txt"):
                WWLLN_Track_Path[dir_indx] = file_paths[file_indx]
                files_found+=1
                
                # generates the name of the storm for images
                Storm_Names[dir_indx] = "Hurricane " + WWLLN_Track_Path[dir_indx].rsplit("_", 3)[1]
                
            elif file_paths[file_indx].endswith("Locations.txt"):
                WWLLN_Locations_Path[dir_indx] = file_paths[file_indx]
                files_found+=1
                
            elif file_paths[file_indx].endswith("Trackfile.csv"):
                Cubic_Spline_Path[dir_indx] = file_paths[file_indx]
                Cubic_Found = True
                files_found+=1
            #print(f"File: {file_paths[file_indx]}, Storm indx: {dir_indx}\n")
            file_indx+=1
        
        # Run with Bens' Cubic Spline
        if not Cubic_Found:
            Cubic_Spline_Path[dir_indx] = join(only_dirs[dir_indx]+"/", Storm_Names[dir_indx] + " Cubic Spline Trackfile.csv")
            #print(WWLLN_Track_Path[dir_indx])
            CubicSplineTrack.cubic_spline_trackfile(WWLLN_Track_Path[dir_indx], Cubic_Spline_Path[dir_indx], check_minutes)
            files_found+=1
        
        if files_found != 3:
            print(f"Incorrect files found: {files_found}, {file_paths}\n")
            only_dirs[dir_indx] = ""
            dir_indx+=1
            continue
        
        # makes the path for this index's storm
        if is_wwlln:
            Image_Path[dir_indx] = join("./Images/", Storm_Names[dir_indx])
        else:
            Image_Path[dir_indx] = "./Images/glm_" + Storm_Names[dir_indx]
        
        if not os.path.exists(Image_Path[dir_indx]):
            os.mkdir(Image_Path[dir_indx])
        
        T_Path[dir_indx] = Image_Path[dir_indx] + "/T Value"
        Image_Path[dir_indx] = Image_Path[dir_indx] + "/" + Storm_Names[dir_indx]
        
        if split_storm:
            Image_Path[dir_indx] = Image_Path[dir_indx] + " Split " + str(check_minutes) +" Minutes"
        else:
            Image_Path[dir_indx] = Image_Path[dir_indx] + " " + str(check_minutes) +" Minutes"
        
        # if an image folder |dosen't| exist, create a new one with the path at Image_Path[dir_indx]
        if not os.path.exists(Image_Path[dir_indx]):
            os.mkdir(Image_Path[dir_indx])
        
        Image_Path[dir_indx] = Image_Path[dir_indx] + '/' # add a slash for use in the next block, making the actual img paths
        
        if not os.path.exists(T_Path[dir_indx]):
            os.mkdir(T_Path[dir_indx])
            T_Path[dir_indx] = T_Path[dir_indx] + '/'
            if not os.path.exists(T_Path[dir_indx]+"0.0"):
                os.mkdir(T_Path[dir_indx]+"0.0")
            T_value = 1.5
            while T_value < 9:
                if not os.path.exists(T_Path[dir_indx]+str(T_value)):
                    os.mkdir(T_Path[dir_indx]+str(T_value))
                T_value+=0.5
        else:
            T_Path[dir_indx] = T_Path[dir_indx] + '/'
        
        #print('\n')
        storms_found+=1
        dir_indx+=1

    if not os.path.exists("Images/T Value"):
        os.mkdir("Images/T Value")
        if not os.path.exists("Images/T Value/0.0"):
            os.mkdir("Images/T Value/0.0")
        T_value = 1.5
        while T_value < 9:
            if not os.path.exists("Images/T Value/"+str(T_value)):
                os.mkdir("Images/T Value/"+str(T_value))
            T_value+=0.5

load_file_paths()
# print for debug
print(f"Storms Found: {storms_found}")
print(Storm_Names)
print(glm_paths)
#print(Image_Path)
#print(WWLLN_Track_Path)
#print(WWLLN_Locations_Path)
#print(Cubic_Spline_Path)

['Data/Arthur', 'Bertha', 'Beta', 'Delta', 'Dolly', 'Edouard']
['Data/Arthur', 'Data/Bertha', 'Beta', 'Delta', 'Dolly', 'Edouard']
['Data/Arthur', 'Data/Bertha', 'Data/Beta', 'Delta', 'Dolly', 'Edouard']
['Data/Arthur', 'Data/Bertha', 'Data/Beta', 'Data/Delta', 'Dolly', 'Edouard']
['Data/Arthur', 'Data/Bertha', 'Data/Beta', 'Data/Delta', 'Data/Dolly', 'Edouard']
['Data/Arthur', 'Data/Bertha', 'Data/Beta', 'Data/Delta', 'Data/Dolly', 'Data/Edouard']
Storms Found: 6
['Hurricane Arthur', 'Hurricane Bertha', 'Hurricane Beta', 'Hurricane Delta', 'Hurricane Dolly', 'Hurricane Edouard']
['Data/Arthur/glm', 'Data/Bertha/glm', 'Data/Beta/glm', 'Data/Delta/glm', 'Data/Dolly/glm', 'Data/Edouard/glm']


In [13]:
"""Team WAAAAAEEEEEEEEE Lance"""
def run_circle(row, center_lat, center_lon): 
    lat = (row['Lat'])
    lon = (row['Long'])
    return great_circle(lat, lon, center_lat, center_lon)

In [14]:
"""Team WAAAAAAEEEEEEEEEE Bhargav"""
#Creating intensity table
intensity_values = {
     'T-Number': [1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6, 6.5, 7, 7.5, 8, 8.5],
  '1-min Winds': [25, 30, 35, 45, 55, 65, 77, 90, 102, 115, 127, 140, 155, 170, 185],
'Min. Pressure': [0, 1009, 1005, 1000, 994, 987, 979, 970, 960, 948, 935, 921, 906, 890, 873]
}
T_frame = pd.DataFrame(intensity_values)

def Tvalue(wind_speed):
    i=0
    half=0
    current = 0
    prev=0
    after=0
    for i in range(15):
        current = T_frame.loc[i, '1-min Winds']
        if (i == 0):
            prev=20
        else:
            prev = T_frame.loc[(i-1), '1-min Winds']
        
        if(i==14):
            after = 188
            
        else:
            after=T_frame.loc[(i+1), '1-min Winds']

        #if(wind_speed < 25):
            #return 1.5
        
        if(wind_speed > 185):
            return 8.5
        
        else:
            half_prev =current - ((current-prev)/2)
            half_aft = current + ((after-prev)/2)
            if(wind_speed >= half_prev and wind_speed < half_aft):
                return T_frame.loc[i, 'T-Number']
    return 0

In [15]:
#------------------------#
" NEW CODE FOR MILESTONE "
#------------------------#
"Lance" 
def get_glm_data_path(day, hour, start_minute, base_path):
    temp_path = base_path + '/' + str(day)
    #print(temp_path)

    start_string = "0.0"
    hour_string = ""
    #end_string = "__.csv"
    if hour < 10:
        hour_string = f"0{hour}"
    else:
        hour_string = f"{hour}"
    temp_path += '/' + hour_string
    
    files_in_path = listdir(temp_path)
    #print(files_in_path)
    file_found = False
    for file in files_in_path:
        if file.startswith(start_string):
            temp_path = join(temp_path +'/', file)
            file_found = True
    
    if not file_found:
        return ""
    
    return temp_path

In [16]:
#------------------------#
" NEW CODE FOR MILESTONE "
#------------------------#
"Bhargava's Code"
#Function to get and arrange generated csv file
def get_csv(csv_file_path):
    temp_csv = pd.read_csv(csv_file_path)
    temp_csv["Date"] = temp_csv["Date"].apply(pd.to_datetime)
    temp_csv= temp_csv.sort_values(["Date"], ascending=True)
    if len(temp_csv["Date"]) == 0:
        return temp_csv
    index = pd.Series(range(int(temp_csv.index.max())+1))
    temp_csv=temp_csv.set_index(index)
    return temp_csv

#------------------------#
" NEW CODE FOR MILESTONE "
#------------------------#
"Cassie"
def load_glm_time_range(date_val, storm_index):
    #print(date_val)
    day = date_val.timetuple().tm_yday
    #print(get_glm_data_path(day, date_val.hour, date_val.minute, glm_paths[storm_index]))
    data_out = [get_csv(get_glm_data_path(day, date_val.hour, date_val.minute, glm_paths[storm_index]))]
    indx = glm_csv_minute_length

    if glm_csv_minute_length >= check_minutes:
        return data_out[0]

    new_date = date_val
    while indx < check_minutes:
        new_date = new_date + datetime.timedelta(minutes=check_minutes)
        day = (new_date - datetime.datetime(new_date.year, 1, 1)).days
        data_out.append(get_csv(get_glm_data_path(day, new_date.hour, new_date.minute, glm_paths[storm_index])))
        indx += glm_csv_minute_length
    
    return pd.concat(data_out)

In [17]:
#-----------------------#
" UPDATED FOR MILESTONE "
#-----------------------#
"""
TODO:
-Clean up code for easier and more clear running & flow
-statitics over a single storm and multiple storms
-Test DBSCAN settings
"""
# for every storm, do the things
"""Team WAAAAAAAEEEEEE Cassie"""
def main():
    disable_ploting = 0
    dir_indx = 0
    while dir_indx < dir_count:
        # if storm is invalid, skip doin the things
        if only_dirs[dir_indx] == "":
            dir_indx+=1
            continue
        
        """Team Squeem"""
        # This loads the WWLLN trackfile Jeremy and Natalia give us
        # wind = nautical mph, pressure = millibars
        df = pd.read_csv(WWLLN_Track_Path[dir_indx], header=None, names=["Year","Month","Day","Hour","Lat","Long","Min_Pressure","Max_Winds","Unused"], low_memory=False, sep='\t')
        df = df.drop("Unused", axis=1)#data filtering
        df['Date'] = df.apply(lambda x: pd.to_datetime(f"{int(x['Year'])}-{int(x['Month'])}-{int(x['Day'])}-{int(x['Hour'])}", format="%Y-%m-%d-%H"), axis=1)
        
        
        "Team WAAAAAEEEEE Lance"
        #wind pressure stuff 
        #run pressure wind speed function on df for every row, save into array
        Tindex = 0
        Tvalues = df['Max_Winds'].apply(Tvalue)
        #Tvalues = [Tvalue(t) for t in df['Max_Winds'] if isdir(join("Data/", d))]
        #make dataframe 
        dates = df['Date']
        #print(Tvalues)
        #print(df['Max_Winds'])
        if len(Tvalues) != len(dates):
            raise Exception("Lengths differ!")
        #loop through df, for every row, check date 
        df_length = len(dates)
        left_half_dates = [date.min] * df_length
        right_half_dates = [date.min] * df_length
        date_index = 1 
        left_half_dates[0] = date.min
        right_half_dates[df_length - 1] = date.max
        while date_index < df_length: 
            left_half_dates[date_index] = dates.iloc[date_index - 1] + ((dates.iloc[date_index] - dates.iloc[date_index - 1]) / 2)
            date_index += 1
        date_index = 0
        while date_index < df_length - 1: 
            right_half_dates[date_index] = dates.iloc[date_index] + ((dates.iloc[date_index] - dates.iloc[date_index + 1]) / 2)
            date_index += 1
        
        """Team Squeem"""
        if is_wwlln:
            # This loads the WWLLN lighting location data that Jeremy and Natalia give us
            ln = pd.read_csv(WWLLN_Locations_Path[dir_indx], header=None, names=["Year","Month","Day","Hour","Min","Sec","Lat","Long","Dist_East_West","Dist_North_South"], low_memory=False, sep=' ')
            ln['Date'] = ln.apply(lambda x: pd.to_datetime(f"{int(x['Year'])}-{int(x['Month'])}-{int(x['Day'])}-{int(x['Hour'])}-{int(x['Min'])}-{x['Sec']}", format="%Y-%m-%d-%H-%M-%S.%f"), axis=1)#is a data clean
        else:
            ln = 0
            # other GLM processing

        # This opens the cubic spline trackfiles that Ben creates
        storm_center = pd.read_csv(Cubic_Spline_Path[dir_indx])
        storm_center['Date'] = storm_center['Date'].apply(pd.to_datetime)

        # Genereates the list of dates from the WWLLN trackfile of a (check_minutes) range
        dates = pd.date_range(df['Date'][0], df['Date'][len(df)-1], freq=str(check_minutes)+"T")
        
        buffer = 10
        if split_storm:
            buffer = 6 # affects map zooooooooooooom
        shp_file = "World_Countries__Generalized_-shp/World_Countries__Generalized_" # Map file ???????
        cmap = 'gist_ncar' # dunno WHAT this is

        # Counter and iteration
        for index, date_val in enumerate(dates): # this loop makes one image for every (check_minutes) amount of time
            starttime = time.perf_counter() # speeed timer start
            center = storm_center[storm_center['Date'] == date_val] # storm center at this time frame
            
            if not is_wwlln:
                ln = load_glm_time_range(date_val, dir_indx)
                #if len(ln):
                    #print(ln)


            """Team WAAAAAAEEEEE Cassie and Lance"""
            if split_storm and is_wwlln:
                ln_curr_timeframe = ln[(ln['Date'] >= date_val) & (ln['Date'] < date_val + datetime.timedelta(minutes=check_minutes))]
                #print(ln_curr_timeframe)
                if len(ln_curr_timeframe) != 0:
                    ln_curr_timeframe['Distance'] = ln_curr_timeframe.apply(run_circle, axis = 1, args = ((center['Lat']), (center['Long'])))
                    #print(ln_curr_timeframe)
                    subset = ln_curr_timeframe[(ln_curr_timeframe['Distance'] <= 100 / 1.852) | ((ln_curr_timeframe['Distance'] >= 200 / 1.852) & (ln_curr_timeframe['Distance'] <= 400 / 1.852))]
                else:
                    subset = ln_curr_timeframe
            else:
                # Select just the lightning locations that are within (check_minutes) minutes
                subset = ln[(ln['Date'] >= date_val) & (ln['Date'] < date_val + datetime.timedelta(minutes=check_minutes))]

            """Team WAAAAAAEEEEEE Cassie"""
            if date_val >= right_half_dates[Tindex]:
                Tindex += 1
            stormTval = Tvalues[Tindex]
            
            # if lightning exists, do dbscan on it
            if len(subset != 0):
                #standard settings: eps=.2, min_samples=3
                dbscan = DBSCAN(eps=.2, min_samples=3)
                subset['Labels'] = dbscan.fit_predict(subset[['Long','Lat']])
            
            """Team Squeem"""
            # Setup the figure
            if not disable_ploting:
                fig, ax = plt.subplots(figsize=(15,15))
            
                # this is for the coastline
                # This reads the shapefile, extracts each shape, creates a polygon, and adds it to the list
                sf = shapefile.Reader(shp_file)
                shapes = sf.shapes()
                Nshp = len(shapes)
                ptchs = []
                for nshp in range(Nshp):
                    pts = np.array(shapes[nshp].points)
                    prt = shapes[nshp].parts
                    par = list(prt) + [pts.shape[0]]
                    for pij in range(len(prt)):
                        ptchs.append(Polygon(pts[par[pij]:par[pij+1]]))
                # Every every polygon from the shapefile to figure
                ax.add_collection(PatchCollection(ptchs, facecolor= '#838688', edgecolor='k', linewidths=1., zorder=2))

                # This adds the water color
                ax.add_patch(mpl.patches.Rectangle((-180,-89),360,180,color='#a6cae0'))

                # This connects Antarcitca to the bottom of the image
                ax.add_patch(mpl.patches.Rectangle((-179.9,-89.9),360,2,color='#838688',zorder=3))

                # These are more general image settings
                if is_wwlln:
                    dataset_name = "WWLLN "
                else:
                    dataset_name = "GLM "
                plt.title(dataset_name + Storm_Names[dir_indx]+"\nT"+str(stormTval)+' '+str(date_val), fontdict = {'fontsize' : 40})
                #edges = (-180,180,-90,90) # Left, Right, Bottom, Top | Set this if you want to change the map scale
                edges = (center['Long'].values[0] - buffer, center['Long'].values[0] + buffer, center['Lat'].values[0] - buffer, center['Lat'].values[0] + buffer) # <--------------------------This affects map zoooooooooom

                # setting up img stuff, prob shouldn't be touched
                xlim = np.append(np.arange(edges[0], edges[1], step=2.5), edges[1])
                ylim = np.append(np.arange(edges[2], edges[3], step=2.5), edges[3])
                ax.set_xticks(xlim)
                ax.set_yticks(ylim)
                ax.set_xlim(edges[0], edges[1])
                ax.set_ylim(edges[2], edges[3])
                ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
                ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
                plt.rc('xtick', labelsize=30)
                plt.rc('ytick', labelsize=30)
            
            """Team WAAAAAEEEEEE Cassie"""
            # if lightning exists, put it in the image
            
            if len(subset) != 0 and not disable_ploting:
                #print(subset)
                plt.scatter(subset['Long'], subset['Lat'], c=subset['Labels'], cmap=cmap, zorder=4) # plots the dbscan data
            
            """Team Squeem"""
            if not disable_ploting:
                #plt.scatter(subset['Long'],subset['Lat'],s=100,c="yellow",edgecolors='black',zorder=5) # <- old plot func for non-grouped data
                plt.scatter(center['Long'], center['Lat'], s=140, c="red", edgecolors='black' ,zorder=5, alpha=0.5) # plots the center of the storm

                # save and close the image
                plt.savefig(Image_Path[dir_indx] + str(index)+".png", bbox_inches='tight', pad_inches=.4)
            if split_storm and not disable_ploting:
                if is_wwlln:
                    shutil.copy(Image_Path[dir_indx] + str(index)+".png", T_Path[dir_indx] + str(stormTval) + '/' + Storm_Names[dir_indx] + ' ' + str(index)+".png")
                    shutil.copy(Image_Path[dir_indx] + str(index)+".png", "Images/T Value/" + str(stormTval) + '/' + Storm_Names[dir_indx] + ' ' + str(index)+".png")
                else:
                    #shutil.copy(Image_Path[dir_indx] + str(index)+".png", T_Path[dir_indx] + str(stormTval) + "/glm_" + Storm_Names[dir_indx] + ' ' + str(index)+".png")
                    shutil.copy(Image_Path[dir_indx] + str(index)+".png", "Images/T Value/" + str(stormTval) + "/glm_" + Storm_Names[dir_indx] + ' ' + str(index)+".png")
            if not disable_ploting:
                plt.close('all')

            # Shows roughly how long we have left
            taken = time.perf_counter() - starttime
            print(f"Storm #{dir_indx}: {Storm_Names[dir_indx]}, T{stormTval}\tPercent Done: {(index + 1) / len(dates) * 100 : .2f}%\tTime taken: {taken : .2f} seconds\tEst time remaining: {datetime.timedelta(seconds=(len(dates) - (index + 1)) * taken)} {len(subset)}", end='\r')
        dir_indx+=1
main()

In [ ]:
"""Team WAAAAAAEEEEE Cassie"""
"""This makes the GIFs"""
dir_indx = 0
while dir_indx < dir_count:
    with imageio.get_writer(Image_Path[dir_indx]+'All.gif', mode='I') as writer:
        Images = [f for f in listdir(Image_Path[dir_indx]) if isfile(join(Image_Path[dir_indx], f)) and f.endswith(".png")]
        img_cnt = len(Images)
        img_indx = 0
        while img_indx < img_cnt:
            image = imageio.imread(Image_Path[dir_indx] + str(img_indx) + ".png")
            writer.append_data(image)
            img_indx+=1
    dir_indx+=1

In [ ]:
#--------------#
" TESTING CELL "
#--------------#
"""
Useful for knowing what columns are in the dataset for coding
appears 'printed' below box, has buttons that can be clicked to show/hide info
"""
from glmtools.io.glm import GLMDataset
glm_data = GLMDataset("C:/Users/GamerPlayer1/Documents/Digipen/PersonalSVN/Spring22/CSP250/Storm_Data/GLM_setup/Dorian/234/00/OR_GLM-L2-LCFA_G16_s20192340000000_e20192340000200_c20192340000226.nc").dataset
glm_data

In [ ]:
#--------------#
" TESTING CELL "
#--------------#
"""
Generic testing code for how to interact with the dataset
"""
from glmtools.io.glm import GLMDataset
glm_data = GLMDataset("C:/Users/GamerPlayer1/Documents/Digipen/PersonalSVN/Spring22/CSP250/Storm_Data/GLM_setup/Dorian/234/00/OR_GLM-L2-LCFA_G16_s20192340000000_e20192340000200_c20192340000226.nc").dataset
groups = glm_data[['group_energy','group_area']]
flashes = glm_data[['flash_energy', 'flash_area']]
# remove unused data (optional)
# groups = groups.drop(['group_parent_flash_id','lightning_wavelength','product_time','group_time_threshold','flash_time_threshold','lat_field_of_view','lon_field_of_view'])
# get the lattitude and longitude values
group_lon, group_lat = groups['group_lon'].values, groups['group_lat'].values
flash_lon, flash_lat = flashes['flash_lon'].values, flashes['flash_lat'].values

(glm_data['number_of_groups'].values).size